<a href="https://colab.research.google.com/github/dlckdtn62/BIO-PREDICTION/blob/master/Compare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path = '/content/drive/My Drive/bio'

In [0]:
import pandas as pd
import numpy as np
import math

In [0]:
test = pd.read_csv(path+'/test.csv')
train = pd.read_csv(path+'/train.csv')
submission=pd.read_csv(path + '/sample_submission.csv', index_col='id')

In [0]:
trn_dst_filled = train.filter(regex='dst$').T.interpolate(method='linear')[::-1].interpolate(method='linear')[::-1].T
tst_dst_filled = test.filter(regex='dst$').T.interpolate(method='linear')[::-1].interpolate(method='linear')[::-1].T

train.update(trn_dst_filled)
test.update(tst_dst_filled)

In [72]:
for row in range(len(train)):
  start, finish = 37, len(train.columns)-5
  for column in range(37, len(train.columns)-4):
    if train.iloc[row, column] == 0:
      for j in range(column, 36, -1):
        if train.iloc[row, j] != 0 and not math.isnan(train.iloc[row, j]):
          start = j
          break
      for j in range(column+1, len(train.columns)-4):
        if train.iloc[row, j] != 0 and not math.isnan(train.iloc[row, j]):
          finish = j
          break
      value = (train.iloc[row, start]+train.iloc[row, finish])/(finish-start+1)
      for j in range(1, finish-start):
        if math.isnan(train.iloc[row, start+j]):
          continue
        train.iloc[row, start+j] = value*j+train.iloc[row, start]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


In [73]:
for row in range(len(test)):
  start, finish = 37, len(test.columns)-1
  for column in range(37, len(test.columns)):
    if test.iloc[row, column] == 0:
      for j in range(column, 36, -1):
        if test.iloc[row, j] != 0 and not math.isnan(test.iloc[row, j]):
          start = j
          break
      for j in range(column+1, len(test.columns)):
        if test.iloc[row, j] != 0 and not math.isnan(test.iloc[row, j]):
          finish = j
          break
      value = (test.iloc[row, start]+test.iloc[row, finish])/(finish-start+1)
      for j in range(1, finish-start):
        if math.isnan(test.iloc[row, start+j]):
          continue
        test.iloc[row, start+j] = value*j+test.iloc[row, start]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in double_scalars
  del sys.path[0]


In [75]:
compare_df = pd.DataFrame()
for i in range(650, 1000, 10):
  compare_df['compare'+'_'+str(i)] = np.log(train[str(i)+'_src']/train[str(i)+'_dst'])

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [0]:
compare_df['rho'] = train['rho']

In [0]:
compare_df = compare_df.replace(math.inf, np.NaN)
compare_df = compare_df.replace(-math.inf, np.NaN)

In [0]:
trn_dst_filled = compare_df.T.interpolate(method='linear')[::-1].interpolate(method='linear')[::-1].T
compare_df.update(trn_dst_filled)

In [0]:
import lightgbm as lgbm
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from lightgbm import LGBMRegressor,  plot_importance

In [0]:
def train_lgbm_model(x_data, y_data, k=5):
    models = []

    kfold = KFold(n_splits=k, shuffle=True, random_state=10)

    for trn_idx, val_idx in kfold.split(x_data):
        x_trn, y_trn = x_data[trn_idx], y_data[trn_idx]
        x_val, y_val = x_data[val_idx], y_data[val_idx]

        trainData = lgbm.Dataset(x_trn, y_trn)
        testData = lgbm.Dataset(x_val, y_val)

        lgb_param = {
            'objective': 'regression', 
            # 'boosting_type': 'dart',
            # 'boosting'  : 'dart',
            'n_estimators': 2000,
            'drop_rate': 0.8, 
            'skip_drop': 0.8, 
            'learning_rate' : 0.03,
            'random_state' : 42,
            'n_jobs' : -1,
            'metric' : 'l1',
            'colsample_bytree' : 0.7,
            'subsample' : 0.7,
            }
        model = lgbm.train(lgb_param, train_set=trainData, num_boost_round=3000, 
                           valid_sets = testData, 
                           verbose_eval=200,
                           early_stopping_rounds=500 # Dart earlystopping 안됨
                           )
        models.append(model)

    return models

In [82]:
models = {}

for label in ['hhb']:
    print("train column : ", label)
    models[label] = train_lgbm_model(np.array(compare_df), train['hhb'])

train column :  hhb
Training until validation scores don't improve for 500 rounds.


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[200]	valid_0's l1: 0.901919
[400]	valid_0's l1: 0.84832
[600]	valid_0's l1: 0.826388
[800]	valid_0's l1: 0.814063
[1000]	valid_0's l1: 0.809291
[1200]	valid_0's l1: 0.807832
[1400]	valid_0's l1: 0.805944
[1600]	valid_0's l1: 0.805326
[1800]	valid_0's l1: 0.804358
[2000]	valid_0's l1: 0.80263
Did not meet early stopping. Best iteration is:
[1980]	valid_0's l1: 0.802526
Training until validation scores don't improve for 500 rounds.
[200]	valid_0's l1: 0.882646
[400]	valid_0's l1: 0.845889
[600]	valid_0's l1: 0.832723
[800]	valid_0's l1: 0.826219
[1000]	valid_0's l1: 0.821105
[1200]	valid_0's l1: 0.817309
[1400]	valid_0's l1: 0.814444
[1600]	valid_0's l1: 0.813605
[1800]	valid_0's l1: 0.81048
[2000]	valid_0's l1: 0.809187
Did not meet early stopping. Best iteration is:
[1994]	valid_0's l1: 0.809113
Training until validation scores don't improve for 500 rounds.
[200]	valid_0's l1: 0.911264
[400]	valid_0's l1: 0.867654
[600]	valid_0's l1: 0.849982
[800]	valid_0's l1: 0.843159
[1000]	valid_

In [0]:
preds = []
for model in models['hhb']:
    preds.append(model.predict(np.array(compare_df)))
answer = []
for j in range(len(preds[0])):
  answer.append(np.mean(np.array(preds)[:, j]))
compare_df['hhb'] = answer

In [84]:
for label in ['hbo2']:
    print("train column : ", label)
    models[label] = train_lgbm_model(np.array(compare_df), train['hbo2'])

train column :  hbo2
Training until validation scores don't improve for 500 rounds.


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[200]	valid_0's l1: 0.55257
[400]	valid_0's l1: 0.527515
[600]	valid_0's l1: 0.518492
[800]	valid_0's l1: 0.512056
[1000]	valid_0's l1: 0.508224
[1200]	valid_0's l1: 0.506159
[1400]	valid_0's l1: 0.504698
[1600]	valid_0's l1: 0.504032
[1800]	valid_0's l1: 0.503286
[2000]	valid_0's l1: 0.502626
Did not meet early stopping. Best iteration is:
[1998]	valid_0's l1: 0.502606
Training until validation scores don't improve for 500 rounds.
[200]	valid_0's l1: 0.566631
[400]	valid_0's l1: 0.544049
[600]	valid_0's l1: 0.536172
[800]	valid_0's l1: 0.531855
[1000]	valid_0's l1: 0.529827
[1200]	valid_0's l1: 0.527731
[1400]	valid_0's l1: 0.526736
[1600]	valid_0's l1: 0.525983
[1800]	valid_0's l1: 0.525867
[2000]	valid_0's l1: 0.525257
Did not meet early stopping. Best iteration is:
[1987]	valid_0's l1: 0.525106
Training until validation scores don't improve for 500 rounds.
[200]	valid_0's l1: 0.554165
[400]	valid_0's l1: 0.534066
[600]	valid_0's l1: 0.525028
[800]	valid_0's l1: 0.519645
[1000]	vali

In [0]:
preds = []
for model in models['hbo2']:
    preds.append(model.predict(np.array(compare_df)))
answer = []
for j in range(len(preds[0])):
  answer.append(np.mean(np.array(preds)[:, j]))
compare_df['hbo2'] = answer

In [86]:
for label in ['ca']:
    print("train column : ", label)
    models[label] = train_lgbm_model(np.array(compare_df), train['ca'])

train column :  ca
Training until validation scores don't improve for 500 rounds.


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[200]	valid_0's l1: 1.84596
[400]	valid_0's l1: 1.73879
[600]	valid_0's l1: 1.68602
[800]	valid_0's l1: 1.65534
[1000]	valid_0's l1: 1.63478
[1200]	valid_0's l1: 1.61952
[1400]	valid_0's l1: 1.6107
[1600]	valid_0's l1: 1.60466
[1800]	valid_0's l1: 1.5998
[2000]	valid_0's l1: 1.59546
Did not meet early stopping. Best iteration is:
[1997]	valid_0's l1: 1.59542
Training until validation scores don't improve for 500 rounds.
[200]	valid_0's l1: 1.92007
[400]	valid_0's l1: 1.79015
[600]	valid_0's l1: 1.72676
[800]	valid_0's l1: 1.68716
[1000]	valid_0's l1: 1.66552
[1200]	valid_0's l1: 1.6518
[1400]	valid_0's l1: 1.64018
[1600]	valid_0's l1: 1.633
[1800]	valid_0's l1: 1.62755
[2000]	valid_0's l1: 1.62404
Did not meet early stopping. Best iteration is:
[1992]	valid_0's l1: 1.62381
Training until validation scores don't improve for 500 rounds.
[200]	valid_0's l1: 1.86175
[400]	valid_0's l1: 1.74474
[600]	valid_0's l1: 1.68948
[800]	valid_0's l1: 1.66112
[1000]	valid_0's l1: 1.64925
[1200]	valid

In [0]:
preds = []
for model in models['ca']:
    preds.append(model.predict(np.array(compare_df)))
answer = []
for j in range(len(preds[0])):
  answer.append(np.mean(np.array(preds)[:, j]))
compare_df['ca'] = answer

In [88]:
for label in ['na']:
    print("train column : ", label)
    models[label] = train_lgbm_model(np.array(compare_df), train['na'])

train column :  na
Training until validation scores don't improve for 500 rounds.


/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[200]	valid_0's l1: 1.29761
[400]	valid_0's l1: 1.26549
[600]	valid_0's l1: 1.24707
[800]	valid_0's l1: 1.2363
[1000]	valid_0's l1: 1.23154
[1200]	valid_0's l1: 1.229
[1400]	valid_0's l1: 1.22698
[1600]	valid_0's l1: 1.22353
[1800]	valid_0's l1: 1.22226
[2000]	valid_0's l1: 1.22189
Did not meet early stopping. Best iteration is:
[1729]	valid_0's l1: 1.22162
Training until validation scores don't improve for 500 rounds.
[200]	valid_0's l1: 1.32476
[400]	valid_0's l1: 1.303
[600]	valid_0's l1: 1.29312
[800]	valid_0's l1: 1.28898
[1000]	valid_0's l1: 1.28701
[1200]	valid_0's l1: 1.28454
[1400]	valid_0's l1: 1.28423
[1600]	valid_0's l1: 1.28483
Early stopping, best iteration is:
[1240]	valid_0's l1: 1.28311
Training until validation scores don't improve for 500 rounds.
[200]	valid_0's l1: 1.2949
[400]	valid_0's l1: 1.26898
[600]	valid_0's l1: 1.25657
[800]	valid_0's l1: 1.24891
[1000]	valid_0's l1: 1.24517
[1200]	valid_0's l1: 1.24298
[1400]	valid_0's l1: 1.24034
[1600]	valid_0's l1: 1.238

In [0]:
preds = []
for model in models['na']:
    preds.append(model.predict(np.array(compare_df)))
answer = []
for j in range(len(preds[0])):
  answer.append(np.mean(np.array(preds)[:, j]))
compare_df['na'] = answer

In [90]:
compare_df_2 = pd.DataFrame()
for i in range(650, 1000, 10):
  compare_df_2['compare'+'_'+str(i)] = np.log(test[str(i)+'_src']/test[str(i)+'_dst'])

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [0]:
compare_df_2 = compare_df_2.replace(math.inf, np.NaN)
compare_df_2 = compare_df_2.replace(-math.inf, np.NaN)

In [0]:
trn_dst_filled = compare_df_2.T.interpolate(method='linear')[::-1].interpolate(method='linear')[::-1].T
compare_df_2.update(trn_dst_filled)

In [0]:
compare_df_2['rho'] = test['rho']

In [0]:
for label in ['hhb', 'hbo2', 'ca', 'na']:
  pred = []
  for model in models[label]:
    pred.append(model.predict(np.array(compare_df_2)))
  answer = []
  for j in range(len(pred[0])):
    answer.append(np.mean(np.array(pred)[:, j]))
  compare_df_2[label] = answer
  submission[label] = answer

In [0]:
submission.to_csv(path+'/just_compared.csv')